# Финальный проект - SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

![https://s3-us-west-2.amazonaws.com/secure.notion-static.com/069818d1-0e5c-4d87-a461-0de584ab9c33/Untitled_(33).png](https://s3-us-west-2.amazonaws.com/secure.notion-static.com/069818d1-0e5c-4d87-a461-0de584ab9c33/Untitled_(33).png)

### Задания

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

### Как выполнить задание?

- Опишите цели исследования;
- Исследуйте таблицы — выведите первые строки;
- Сделайте по одному SQL-запросу для решения каждого задания;
- Выведите результат каждого запроса в тетрадке;
- Опишите выводы по каждой из решённых задач.

In [20]:
import pandas as pd
from sqlalchemy import create_engine

db_config = {
    'user': 'praktikum_student',
    'pwd': 'Sdf4$2;d-d30pp',
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432,
    'db': 'data-analyst-final-project-db'
}

connection_string = (
    'postgresql://{}:{}@{}:{}/{}'
    .format(
        db_config['user'],
        db_config['pwd'],
        db_config['host'],
        db_config['port'],
        db_config['db']
    )
)

engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Исходные данные

In [21]:
for df in ['books', 'authors', 'publishers', 'reviews']:
    print('Dataframe', df)
    query = (
    '''
    SELECT *
    FROM
    ''' 
    + df + 
    '''
    LIMIT 3
    '''
    )
    display(pd.io.sql.read_sql(query, con = engine))
    
    

Dataframe books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135


Dataframe authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie


Dataframe publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books


Dataframe reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...


## Задания

Посчитайте, сколько книг вышло после 1 января 2000 года

In [22]:
query = (
    '''
    SELECT COUNT(book_id)
    FROM books
    WHERE publication_date >= DATE('2000-01-01')
    '''
)
pd.io.sql.read_sql(query, con = engine)

,count
0,821


Для каждой книги посчитайте количество обзоров и среднюю оценку

In [31]:
query = (
    '''
    SELECT b.title ,
        count(b.book_id) AS number_of_reviews,
        avg(rating) AS avg_rating
    FROM books AS b
    LEFT JOIN ratings AS r ON b.book_id=r.book_id
    GROUP BY b.title
    ORDER BY 3 DESC
    '''
)
pd.io.sql.read_sql(query, con = engine)

,title,number_of_reviews,avg_rating
0,Captivating: Unveiling the Mystery of a Woman'...,2,5.00
1,Evening Class,2,5.00
2,In the Hand of the Goddess (Song of the Liones...,3,5.00
3,The Big Bad Wolf (Alex Cross #9),2,5.00
4,A Dirty Job (Grim Reaper #1),4,5.00
...,...,...,...
994,The World Is Flat: A Brief History of the Twen...,4,2.25
995,His Excellency: George Washington,2,2.00
996,Junky,2,2.00
997,Drowning Ruth,3,2.00


Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.

In [32]:
query = (
    '''
    SELECT p.publisher,
         count(b.book_id)
    FROM books AS b
    LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
    WHERE b.num_pages > 50
    GROUP BY p.publisher
    ORDER BY count(b.book_id) DESC limit 1
    '''
)
pd.io.sql.read_sql(query, con = engine)

,publisher,count
0,Penguin Books,42


Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.

In [33]:
query = (
    '''
    SELECT a.author,
            round(avg(tabl_book.avg_raiting),
            2) FROM
        (SELECT title,
            AVG(rating) AS avg_raiting,
            COUNT(rating) AS cnt
        FROM books
        LEFT JOIN ratings
        ON books.book_id = ratings.book_id
        GROUP BY title
        HAVING COUNT(rating_id) >= 50) AS tabl_book
    LEFT JOIN books AS b
        ON b.title = tabl_book.title
    LEFT JOIN authors AS a
        ON a.author_id = b.author_id
    GROUP BY a.author
    ORDER BY avg(tabl_book.avg_raiting) DESC LIMIT 1
    '''
)
pd.io.sql.read_sql(query, con = engine)

,author,round
0,J.K. Rowling/Mary GrandPré,4.28


Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [35]:
query = (
    '''
    SELECT round(avg(tabl.count),
            0) AS avg_count from
        (SELECT r.username,
            count(r.username) from
            (SELECT username from
                (SELECT username,
            count(username) AS count_estimation
                FROM ratings
                GROUP BY  username) AS count_username
                WHERE count_username.count_estimation > 50) AS us
                LEFT JOIN reviews AS r
                    ON us.username = r.username
                GROUP BY r.username) AS tabl
    '''
)
pd.io.sql.read_sql(query, con = engine)

,avg_count
0,24.0


## Вывод

По базе данных выполнили задания:

1. Посчитайте, сколько книг вышло после 1 января 2000 года - 821 книга
2. Для каждой книги посчитайте количество обзоров и среднюю оценку - см. таблицу выше
3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц - издательство Penguin Books выпустило 42 книги
4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками - автор J.K. Rowling/Mary GrandPré, средняя оценка 4.28
5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок - среднее количество обзоров 24.33